In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [2]:
import matplotlib.pyplot as plt

In [3]:
# data loading in appropriate formate

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


11501568/11490434 [==============================] - 0s 0us/step


In [4]:
y_train=tf.keras.utils.to_categorical(y_train)

In [5]:
class CNNBlock(tf.keras.layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(out_channels, kernel_size, padding="same")
        self.max = tf.keras.layers.MaxPooling2D(kernel_size)

    def call(self, input_tensor):
        x = self.conv(input_tensor)
        x = tf.nn.relu(x)
        x = self.max(x)
        return x

In [7]:
#model subclassing with convolution layer.

class MyModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        # define all layers in init
        # Layer of Block 1
        self.c1 = CNNBlock(32)

        # Layer of Block 2
        self.c2 = CNNBlock(64)

        #fully connected layer
        self.flat = tf.keras.layers.Flatten()
        self.d1 = keras.layers.Dense(20, activation='relu')
        self.d2 = keras.layers.Dense(num_classes)


    def call(self, input_tensor):
        # forward pass: block 1 
        x = self.c1(input_tensor)
      
        # forward pass: block 2 
        x = self.c2(x)

        # forward pass: FC layer
        x=self.flat(x)
        x = self.d1(x)
        return self.d2(x)


' def model(self):\n    x = keras.Input(shape=(28, 28, 1))\n    return keras.Model(inputs=[x], outputs=self.call(x)) '

In [9]:
model=MyModel(10)

"""loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])"""
model.compile(
          loss      = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
          metrics   = tf.keras.metrics.CategoricalAccuracy(),
          optimizer = tf.keras.optimizers.Adam())

In [10]:
model.build(input_shape=(None, 28, 28, 1))
model.call(tf.keras.Input(shape=(28, 28, 1)))
model.summary()
#no of parameter in conv  = ((m*n*d)+1)*k) for ex ((3*3*1)+1)*32 = 320
# ((3*3*32)+1)*64) = 18496

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_block (CNNBlock)        (None, 9, 9, 32)          320       
                                                                 
 cnn_block_1 (CNNBlock)      (None, 3, 3, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 576)               0         
                                                                 
 dense (Dense)               (None, 20)                11540     
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
Total params: 30,566
Trainable params: 30,566
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(x_train, y_train, batch_size=128,epochs=2,validation_split=0.2)

Epoch 1/2
375/375 [==============================] - 39s 101ms/step - loss: 0.5581 - categorical_accuracy: 0.8240 - val_loss: 0.1440 - val_categorical_accuracy: 0.9597
Epoch 2/2
375/375 [==============================] - 36s 95ms/step - loss: 0.1262 - categorical_accuracy: 0.9613 - val_loss: 0.0960 - val_categorical_accuracy: 0.9708
